In [ ]:
# IMPORTS
import datetime as dt
import os
import sys
import warnings

from pathlib import Path
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from river import anomaly, preprocessing, utils
from river.metrics import F1, MacroF1
from river.metrics.base import BinaryMetric
from sklearn.decomposition import PCA

sys.path. insert(1, str(Path().resolve().parent))
from functions.anomaly import ConditionalGaussianScorer
from functions.compose import build_model, convert_to_nested_dict
from functions.proba import MultivariateGaussian
from functions.evaluate import print_stats, progressive_val_predict

# CONSTANTS
RANDOM_STATE = 42

# FUNCTIONS
def tune_train_model(steps, df, val_kwargs: dict = {}, **params):
    params = convert_to_nested_dict(params)
    model = build_model(steps, params)
    metric: BinaryMetric = F1()
    try:
        val_kwargs.update(params.get("Val", {}))
        progressive_val_predict(model, df, [metric], print_every=0,  print_final=False, **val_kwargs)

        return metric.get()
    except Exception as e:
        print(e)
        return 0


def get_random_samples(df: pd.DataFrame, num_samples=10000):
    if len(df) <= num_samples:
        return df
    else:
        return df.sample(n=num_samples, random_state=RANDOM_STATE)


def plot_detection(df: pd.DataFrame, y_pred):
    df['pred'] = y_pred
    if 'anomaly' in df.columns:
        df = get_random_samples(df)
        if len(df.columns) >= 4:
            # Separate the feature columns from the target column ("anomaly")
            X = df.drop(columns=['anomaly', 'pred'])
            y = df['anomaly']
            y_pred = df['pred']

            # Apply PCA to reduce the feature columns to 2 components
            pca = PCA(n_components=2)
            X_pca = pca.fit_transform(X)

            # Create a new DataFrame with the reduced components and "anomaly" column
            df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
            df_pca['anomaly'] = y.values
            df_pca['pred'] = y_pred.values
        else:
            print(True)
            df_pca = pd.DataFrame(
                df.reset_index().copy()
                )
            df_pca.columns = ['PC1', 'PC2', 'anomaly', 'pred']

        # Plot the 2D scatter plot
        plt.scatter(df_pca[df_pca['anomaly'] == 0]['PC1'], df_pca[df_pca['anomaly'] == 0]['PC2'])
        plt.scatter(df_pca[df_pca['anomaly'] == 1]['PC1'], df_pca[df_pca['anomaly'] == 1]['PC2'], facecolors='none', edgecolors='r', linewidths=0.5)
        plt.scatter(df_pca[df_pca['pred'] == 1]['PC1'], df_pca[df_pca['pred'] == 1]['PC2'], marker='x', linewidths=1)  # type: ignore
        plt.xticks(())
        plt.yticks(())


def save_results_y(df_ys, path):
    dir_path = path
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    df_ys.to_csv(f"{dir_path}/ys.csv", index=False)


def save_results_metrics(metrics_res, path):
    dir_path = path
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    metrics_res.to_csv(f"{dir_path}/metrics.csv")

# MODS
class QuantileFilter(anomaly.QuantileFilter):
  def __init__(self, anomaly_detector, q: float, protect_anomaly_detector=True):
        super().__init__(
            anomaly_detector=anomaly_detector,
            protect_anomaly_detector=protect_anomaly_detector,
            q=q
        )
  def predict_one(self, *args):
    score = self.score_one(*args)
    return self.classify(score)

# SETTINGS

# DETECTION ALGORITHMS
detection_algorithms = [
    (   
        "Conditional Gaussian Scorer",
        [[ConditionalGaussianScorer,[utils.Rolling, MultivariateGaussian]]],
        {
            "ConditionalGaussianScorer__threshold": (0.85, 0.99994),
            "Rolling__window_size__round": (150 , 10000),
            "ConditionalGaussianScorer__t_a__int": (50, 2000),
            "ConditionalGaussianScorer__grace_period__round": (50, 1000),
            }
        ),
    (   
        "Half-Space Trees",
        [preprocessing.MinMaxScaler, [QuantileFilter, anomaly.HalfSpaceTrees]],
        {
            "QuantileFilter__q": (0.85, 0.99994),
            "HalfSpaceTrees__n_trees__round": (1, 20),
            "HalfSpaceTrees__height__round": (2, 14),
            "HalfSpaceTrees__window_size__round": (100, 400)
            }
        ),
    (   "One-Class SVM",
        [preprocessing.StandardScaler, [QuantileFilter, anomaly.OneClassSVM]],
        {
            "QuantileFilter__q": (0.85, 0.99994),
            "OneClassSVM__intercept_lr": (0.005, 0.02),
            }
        ),
    # (   "Local Outlier Factor",
    #     [[QuantileFilter, anomaly.LocalOutlierFactor]],
    #     {
    #         "QuantileFilter__q": (0.85, 0.99994),
    #         "LocalOutlierFactor__n_neighbors__round": (0,200),
    #         }
    #     ),
]

# DATASETS
datasets = [
    # {
    #     "name": "Load Balancing",
    #     "data": pd.read_csv(
    #         "data/load_balancing.csv", index_col=0).dropna(axis=0),
    #     "anomaly_col": "anomaly",
    #     "drop": "MPC:Request Status Code"},
    {
        "name": "SKAB",
        "data": pd.read_csv(
            "data/multivariate/alldata_skab.csv",
            index_col=0).dropna(axis=0),
        "anomaly_col": "is_anomaly",
        "drop": "changepoint"},
    # {
    #     "name": "YAHOO",
    #     "data": pd.read_csv(
    #         "data/multivariate/yahoo_sub_5.csv",
    #         index_col=0).dropna(axis=0),
    #     "anomaly_col": "is_anomaly",
    #     "drop": None},
    # {
    #     "name": "Room Occupancy",
    #     "data": pd.read_csv(
    #         "data/multivariate/Occupancy/room-occupancy-1.test.csv",
    #         index_col=0).dropna(axis=0),
    #     "anomaly_col": "is_anomaly",
    #     "drop": None},
    # {
    #     "name": "Archive",
    #     "data": pd.read_csv(
    #         "data/multivariate/archive/TimeSeries.csv"
    #         ).head(80000).tail(20000),
    #     "anomaly_col": pd.read_csv(
    #         "data/multivariate/archive/labelsTimeSeries.csv"
    #         ).head(80000).tail(20000)['label'],
    #     "drop": None},
]

# PLOT CONFIG
plt.figure(figsize=(len([1,1,1]) * 2 + 4, 12.5))
plt.subplots_adjust(
    left=0.02, right=0.98, bottom=0.001, top=0.96, wspace=0.05, hspace=0.01
)
plot_num = 1

# RUN
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for dataset in datasets:
        # PREPROCESS DATA
        df = dataset["data"]
        df.index = pd.to_timedelta(
            range(0, len(df)), 'T') + pd.Timestamp.now().replace(microsecond=0)
        if isinstance(dataset["anomaly_col"], str):
            df = df.rename(columns={dataset["anomaly_col"]: "anomaly"})
        elif isinstance(dataset["anomaly_col"], pd.Series):
            df_y = dataset["anomaly_col"]
            df['anomaly'] = df_y.rename('anomaly').values
        if dataset["drop"] is not None:
            df = df.drop(columns=dataset["drop"])
        print(f"\n=== {dataset['name']} === [{sum(df['anomaly'])}/{len(df)}]"
            .ljust(80, '='))

        df_ys = df[["anomaly"]].copy()
        # RUN EACH MODEL AGAINST DATASET
        for alg in detection_algorithms:
            print(f"\n===== {alg[0]}".ljust(80, '='))
            # INITIALIZE OPTIMIZER
            pbounds = alg[2]
            mod_fun = partial(tune_train_model, alg[1], df, {})
            if len(alg) == 4:
                kwargs = alg[3]
            else:
                kwargs = {}
            # INITIALIZE METRICS
            metrics_list = []

            # TUNE HYPERPARAMETERS
            optimizer = BayesianOptimization(
                f=mod_fun,
                pbounds=pbounds,
                verbose=2,
                random_state=RANDOM_STATE,
                allow_duplicate_points=True
            )
            logger = JSONLogger(path=f"./.results/{dataset['name']}-{alg[0]}.log")
            optimizer.subscribe(Events.OPTIMIZATION_END, logger)
            optimizer.maximize(init_points=1, n_iter=5)
            params = convert_to_nested_dict(optimizer.max["params"])
            kwargs.update(params.get("Val", {}))
            print(params)
            model = build_model(alg[1], params)
            # USE TUNED MODEL
            # PROGRESSIVE PREDICT
            y_pred, change_point, _, _ = (
                progressive_val_predict(model, df, metrics=None, print_every=0,
                                        **kwargs))
            
            # SAVE PREDICITONS
            df_ys[f"{alg[0]}__{params}"] = y_pred
        
            #  PRINT OUT LAST DETECTION RESULTS
            print_stats(df, y_pred, change_point)
            plt.subplot(len(datasets), len(detection_algorithms), plot_num)
            plot_detection(df, y_pred)
            plot_num +=1
        
        # LOAD RESULTS
        #  Save
        dir_path = f".results/{dataset['name']}"
        save_results_y(df_ys,
                        f".results/{dataset['name']}")

plt.show()

In [3]:
import os

import pandas as pd

from river.metrics import F1, Precision, Recall, ClassificationReport

metrics = [
    Precision(), Recall(), F1(),
    ClassificationReport(),
]

# show folders in dir
for folder in os.listdir(".results"):
    # check if listed object is a folder and does not start with a period
    if os.path.isdir(os.path.join(".results", folder)) and not folder.startswith('.'):
        # loop through the files in the folder
        for file in os.listdir(os.path.join(".results", folder)):
            if file == "ys.csv":
                df_ys = pd.read_csv(
                    os.path.join(".results", folder) + "/ys.csv")
                df_metrics = pd.DataFrame(index=[
                    'Precision', 'Recall', 'F1',
                    'MacroPrecision', 'MacroRecall', 'MacroF1',
                    'WeightedPrecision', 'WeightedRecall', 'WeightedF1', 'FAR',
                    'Support_False', 'Support_True'])
                df_stats = df_metrics.copy()
                for col in df_ys.columns[1:]:
                    
                    metrics_ = [metric.clone() for metric in metrics]
                    for y_true, y_pred in zip(df_ys.anomaly, df_ys[col]):
                        
                        for metric in metrics_:
                            metric = metric.update(y_true, y_pred)
                    cr = metrics_.pop(-1)
                    cm = cr.cm
                    df_metrics[col] = [
                        metric.get() for metric in metrics_] + [
                        cr._macro_precision.get(),
                        cr._macro_recall.get(),
                        cr._macro_f1.get(),
                        cr._weighted_precision.get(),
                        cr._weighted_recall.get(),
                        cr._weighted_f1.get(),
                        cm.total_false_positives / (cm.total_false_positives + cm.total_true_negatives),
                        *cm.sum_row.values()
                        ]
                        
                col_pat = df_metrics.columns.str.split('__', n=2).str[0]
                df_metrics_grouped = df_metrics.groupby(col_pat, axis=1)
                group_idx = pd.Index(df_metrics_grouped.groups.keys())
                df_stats[group_idx+"_max"] = df_metrics_grouped.max()
                df_stats[group_idx+"_mean"] = df_metrics_grouped.mean()
                df_stats[group_idx+"_std"] = df_metrics_grouped.std()
            
                dir_path = f".results/{folder}"
                df_metrics.to_csv(f"{dir_path}/metrics.csv")
                df_stats.to_csv(f"{dir_path}/stats.csv")
